In [45]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy import signal
import seaborn as sns
from scipy.stats import spearmanr

In [2]:
import scikit_posthocs as sp
import numpy as np
from scipy import stats
import statsmodels.api as sm
import json

In [3]:
import warnings
warnings.filterwarnings('ignore')

# 3. Hypothesis Testing

In [19]:
Daily_level_Data=pd.read_csv("Data_Daily_level.csv")
#Drop ROAS (estimated_smoothened) as it is the expected valuie and we want to perform hypothesis testing
#on ROAS (up to Date)
Daily_level_Data.drop(columns=["ROAS (estimated)_smoothened"],inplace=True)

In [20]:
Daily_level_Data.head()

,Date_Daily_Level,Average Signups,Active %,CPA,Average observed profit,Estimated LTV,New Users,Users,Bounces,Pageviews,Page Load Time (ms),ROAS (up to date)_smoothened
0,2021-08-01,1101,0.022707,31.362341,136.178029,143.416200,NaN,NaN,NaN,NaN,NaN,4.232135
1,2021-08-02,802,0.017456,39.260330,143.763778,149.335326,NaN,NaN,NaN,NaN,NaN,4.055284
2,2021-08-03,929,0.020452,33.946871,141.372034,147.895183,NaN,NaN,NaN,NaN,NaN,3.906417
3,2021-08-04,910,0.019780,36.023701,149.599429,155.909300,NaN,NaN,NaN,NaN,NaN,3.863493
4,2021-08-05,1025,0.015610,38.087794,140.391415,145.375581,NaN,NaN,NaN,NaN,NaN,3.954360


### Hypothesis 1

**Hypothesis to check** :  Decline in ROAS is linkd with increase in the average page load time
<br>**Null Hypothesis** : There in is no relation between ROAS and associated average page load time

In [28]:
hypothesis_1=Daily_level_Data[~Daily_level_Data["Page Load Time (ms)"].isna()][["Page Load Time (ms)","ROAS (up to date)_smoothened"]]

In [29]:
correlation = hypothesis_1['Page Load Time (ms)'].corr(hypothesis_1['ROAS (up to date)_smoothened'])
print(f"Correlation between Page Load Time and ROAS: {correlation}")

Correlation between Page Load Time and ROAS: -0.07610040434295756


* There is weak correlation between both variables

In [34]:
X = hypothesis_1['Page Load Time (ms)']
X = sm.add_constant(X)  
y = hypothesis_1['ROAS (up to date)_smoothened']
model = sm.OLS(y, X).fit()

In [35]:
print(model.summary())

                                 OLS Regression Results                                 
Dep. Variable:     ROAS (up to date)_smoothened   R-squared:                       0.006
Model:                                      OLS   Adj. R-squared:                  0.004
Method:                           Least Squares   F-statistic:                     3.349
Date:                          Mon, 28 Aug 2023   Prob (F-statistic):             0.0677
Time:                                  21:51:36   Log-Likelihood:                -492.47
No. Observations:                           577   AIC:                             988.9
Df Residuals:                               575   BIC:                             997.7
Df Model:                                     1                                         
Covariance Type:                      nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
----------------------

**At a Threshold significane level of 0.05** : there is no enough evidence to reject Null Hypothesis. 

### Hypothesis 2

**Hypothesis Formulation** :  Decline in ROAS is linked to decrease in the percentage of active users
<br>**Null Hypothesis** : There is no relation between ROAS and percentage of active users

In [41]:
hypothesis_2=Daily_level_Data[~Daily_level_Data["Active %"].isna()][["Active %","ROAS (up to date)_smoothened"]]

In [49]:
#Apply Spearmann Correlation coefficient
correlation, p_value = spearmanr(hypothesis_2['Active %'], hypothesis_2['ROAS (up to date)_smoothened'])
print(f"P-value: {p_value}")

P-value: 8.488550043641476e-33


**At a P-value Threshold significance level of 0.05 :** there is enough evidence to reject Null Hypothesis and validate statistical significance between ROAS and Percentage of active useres.

**Possible Reasons that can cause decrease in percentage of active users that is worth investigation is**:
1. Unappealing Website Content
2. Poor User Experience
3. Drop in product relevenace to existing customers

### Hypothesis 3

**Hypothesis Formulation** : Decline in ROAS is linked to increase in CPA
<br>**Null Hypothesis** : There is no association between CPA and ROAS

In [50]:
hypothesis_3=Daily_level_Data[~Daily_level_Data["Active %"].isna()][["CPA","ROAS (up to date)_smoothened"]]

In [52]:
# Correlation analysis between CPA and ROAS
correlation, p_value = stats.pearsonr(hypothesis_3["CPA"], hypothesis_3["ROAS (up to date)_smoothened"])
print(f"P-value: {p_value}")

P-value: 4.311456921135876e-95


**At a P-value Threshold significance level of 0.05 :** there is enough evidence to reject Null Hypothesis and validate statistical significance between ROAS and CPA (Cost Per Accquisition).

**Possible Reasons that can cause higher CPA that is worth investigation is**:
1. Higher Advertising Costs
2. Increased Competition among competitors
3. Less Effective Targeting Strategy

In [11]:
Monthly_level_Data=pd.read_csv("Data_Monthly_level.csv")
for column in Monthly_level_Data.columns[1:]:
    Monthly_level_Data.loc[:,column]=Monthly_level_Data[column].apply(lambda x:x.strip('][').replace("'","").split(','))
    Monthly_level_Data.loc[:,column]=Monthly_level_Data[column].apply(lambda x:[temp.strip() for temp in x])
    try:
        Monthly_level_Data.loc[:,column]=Monthly_level_Data[column].apply(lambda x:[float(temp) for temp in x])
    except:
        pass
Monthly_level_Data.head(2)

,Date_Monthly_Level,Operating System,Region,Users_1,Bounces_1,Sessions_1,Pageviews_1,Page Load Time (ms)_1,Signups_1,Source / Medium,Users_3,Bounces_3,Sessions_3,Pageviews_3,Page Load Time (ms)_3,Signups_3
0,2021-09-01,"[iOS, Android, iOS, iOS, iOS, iOS, iOS, iOS, i...","[Texas, Texas, California, Florida, Georgia, N...","[28670.0, 15790.0, 14785.0, 14520.0, 13700.0, ...","[18461.0, 9601.0, 8490.0, 7697.0, 8014.0, 6691...","[43508.0, 27110.0, 23830.0, 20550.0, 19307.0, ...","[124863.0, 105767.0, 73553.0, 59694.0, 55436.0...","[5045524.0, 6669434.0, 2814674.0, 1961975.0, 1...","[1798.0, 1137.0, 397.0, 661.0, 714.0, 714.0, 7...","[fb / paidsocial, (direct) / (none), m.faceboo...","[170831.0, 90031.0, 74029.0, 34436.0, 33431.0,...","[98944.0, 46391.0, 39355.0, 19413.0, 19307.0, ...","[189927.0, 179744.0, 83128.0, 72971.0, 60117.0...","[365730.0, 804695.0, 173899.0, 336213.0, 24782...","[26251890.0, 10469393.0, 8633498.0, 7425305.0,...","[11399.0, 1878.0, 6268.0, 1587.0, 2248.0, 1719..."
1,2021-10-01,"[iOS, Android, iOS, iOS, iOS, iOS, iOS, Androi...","[Texas, Texas, North Carolina, Georgia, Califo...","[24385.0, 17800.0, 11637.0, 11346.0, 11082.0, ...","[13542.0, 10209.0, 6850.0, 6877.0, 6162.0, 629...","[36975.0, 28326.0, 18091.0, 16319.0, 17165.0, ...","[120367.0, 99552.0, 54775.0, 49009.0, 52844.0,...","[5250262.0, 1053655.0, 2683780.0, 1331258.0, 1...","[1375.0, 1349.0, 397.0, 450.0, 582.0, 661.0, 7...","[fb / paidsocial, (direct) / (none), m.faceboo...","[140574.0, 92279.0, 56388.0, 39382.0, 39329.0,...","[78975.0, 47792.0, 29940.0, 14811.0, 22005.0, ...","[156945.0, 178395.0, 63979.0, 44090.0, 84397.0...","[311193.0, 804298.0, 125974.0, 113226.0, 40754...","[14928369.0, 20728177.0, 4494073.0, 7272989.0,...","[9759.0, 1878.0, 3571.0, 3385.0, 2592.0, 1851...."
